In [10]:
# -*- coding: utf-8 -*-
#библиотеки
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [11]:
import sklearn
print('Sklearn version:',sklearn.__version__)
print('pandas version:',pd.__version__)
print('numpy version:',np.__version__)

Sklearn version: 0.23.2
pandas version: 1.1.1
numpy version: 1.18.5


In [12]:
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize
#открываем тестовые данные
txt = open(r'test.csv', encoding='utf-8').read()
sents = sent_tokenize(txt)

In [13]:
sents

['<review>Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться.',
 'Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню.',
 'Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.',
 'Несколько неудобно что нету отдельной кнопки для фото.',
 'Подумываю купить батарею большей емкость мб что нибудь измениться.',
 '</review>\n\n<review>ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если \nпостоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается.',
 'Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -

In [14]:
#приводим данные в удобный вид
import re;
b=[]
c=[]
flag=0
sents2=[]
test=[]
for line in sents:   
    if re.search(r'<review>', line) and re.search(r'</review>', line):
        l=line.split('\n\n')
        for i in l:
            sents2.append(i)
    else:
        sents2.append(line)

for line in sents2:   
    if re.search(r'<review>', line):
        flag=1;
    if flag==1:
        b.append(line)
    if re.search(r'</review>', line):
        flag=0
        full_otziv=' '.join(b)
        c.append(full_otziv)
        b=[] 

for line in c:
    test.append(line.replace('<review>','').replace('</review>','').replace('\n',''))

In [15]:
test

['Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти. Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться. ',
 'ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если постоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается. Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор смс на обычных сенсорниках и кнопо

In [16]:
#считываем обучающий датасет (спарсенный)
a=pd.read_csv('data_otziv.csv')
#делим на отзывы и классы
X=a['otziv']
y=a['target']

In [17]:
a

,otziv,target
0,1. Батарея - с утра до вечера хватит при любом...,1
1,"Экран, надёжный, качество,не глючный, звук отл...",1
2,За такие деньги - сумасшедшая производительнос...,1
3,Отличный смартфон для своей цены греется при...,1
4,"Держит заряд 3 дня, если мало пользоваться 3G ...",1
...,...,...
5193,"Однозначно экран, большой и яркий с оптимальны...",1
5194,"Сразу напишу про дизайн, впервые увидев фото п...",1
5195,"красивый,легкий,прочный,офигенный дисплей,каме...",1
5196,"Легкий Тонкий Отличный экран,пиксилей не видно...",1


In [18]:
#опять библиотеки
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

In [19]:
#делим на обучающую и тестовую выборку
X_train, X_test, y_train, y_test =\
    train_test_split(X,y, test_size=0.1, random_state=2020)
#линейный классификатор
classific = LinearSVC(random_state=2020,max_iter=10000, tol=0.0001, class_weight='balanced')

vectorizer=TfidfVectorizer(max_df=0.85, min_df=0.003, ngram_range=(1, 3))
#строим пайплайн
pipe = Pipeline([("transformer",vectorizer),
                 ("classifier",classific)])
#обучаем модель
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
#неплохая точность. больше 85
roc_auc_score(y_test, y_pred)

0.8531466290694694

In [20]:
#обучаем на всей выборке
pipe.fit(X, y)
#заполняем массив как нам сказали в кегле и записываем его
ident = [i for i in range(0,len(test))]
result=pd.DataFrame({'Id': ident})
target=pipe.predict(test) #находим классы для теста
target_bukva=np.where(target==0,'neg','pos')
result['y']=target_bukva
result[['Id','y']].to_csv('keglya2.csv', index = False)

In [21]:
#выгружаем классификатор и векторизатор
import pickle
f = open('class.pkl', 'wb')
pickle.dump(classific, f)
f.close()
f = open('vect.pkl', 'wb')
pickle.dump(vectorizer, f)
f.close()
